# **EDA**

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
from google.colab import drive

In [3]:
drive.mount('//content/drive')

Drive already mounted at //content/drive; to attempt to forcibly remount, call drive.mount("//content/drive", force_remount=True).


In [4]:
dataset_path=os.listdir('/content/drive/MyDrive/image-classifier/ear_wear')

In [5]:
print(dataset_path)

['earphone', 'headphone', 'phone']


In [6]:
print('type of classes to classify', len(dataset_path))

type of classes to classify 3


In [7]:
class_labels=[]

for item in dataset_path:
  all_classes=os.listdir('/content/drive/MyDrive/image-classifier/ear_wear'+'/'+item)

  for room in all_classes:
      class_labels.append((item,str('dataset_path' +'/'+ item)+ '/' + room))

In [8]:
df=pd.DataFrame(data=class_labels,columns=['labels','image'])

In [9]:
df

,labels,image
0,earphone,dataset_path/earphone/e2.png
1,earphone,dataset_path/earphone/e1.jpg
2,earphone,dataset_path/earphone/e3.png
3,earphone,dataset_path/earphone/e4.png
4,earphone,dataset_path/earphone/e5.png
5,earphone,dataset_path/earphone/e6.jpg
6,earphone,dataset_path/earphone/e7.jpg
7,earphone,dataset_path/earphone/e8.jpg
8,earphone,dataset_path/earphone/e9.jpg
9,earphone,dataset_path/earphone/e10.jpg


In [10]:
print('totl number of images',len(df))
df['labels'].value_counts()

totl number of images 30


earphone     10
headphone    10
phone        10
Name: labels, dtype: int64

In [11]:
import cv2
path= '/content/drive/MyDrive/image-classifier/ear_wear/'

dataset_path= os.listdir('/content/drive/MyDrive/image-classifier/ear_wear')
im_size=244
images=[]

labels=[]

for i in dataset_path:
  data_path= path + str(i)

  filenames=[i for i in os.listdir(data_path) ]

  for f in filenames:
    img=cv2.imread(data_path +'/'+f)
    img=cv2.resize(img,(244,244))
    images.append(img)
    labels.append(i)


In [12]:
images=np.array(images)

In [13]:
images=images.astype('float32')/255.0

In [14]:
images.shape

(30, 244, 244, 3)

In [15]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [16]:
y=df['labels'].values

In [17]:
print(y)

['earphone' 'earphone' 'earphone' 'earphone' 'earphone' 'earphone'
 'earphone' 'earphone' 'earphone' 'earphone' 'headphone' 'headphone'
 'headphone' 'headphone' 'headphone' 'headphone' 'headphone' 'headphone'
 'headphone' 'headphone' 'phone' 'phone' 'phone' 'phone' 'phone' 'phone'
 'phone' 'phone' 'phone' 'phone']


In [18]:
le=LabelEncoder()
y=le.fit_transform(y)

In [19]:
print(y)

[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2]


In [20]:
y=y.reshape(-1,1)


In [21]:
from sklearn.compose import ColumnTransformer
ct=ColumnTransformer([('my_ohe',OneHotEncoder(), [0])],remainder='passthrough')

In [22]:
Y=ct.fit_transform(y)

In [23]:
print(Y)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [24]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

imges,Y=shuffle(images,Y,random_state=1)

In [25]:
x_train,x_test,y_train,y_test=train_test_split(images,Y,test_size=0.50,random_state=200)

In [26]:
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)


(15, 244, 244, 3) (15, 244, 244, 3) (15, 3) (15, 3)


# EFFICIENTNET 

In [33]:
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0

In [34]:
NUM_CLASSES=3
IMG_SIZE=244
size=(244,244)

In [35]:

efficientnet = EfficientNetB0(weights='imagenet', include_top=False)

In [37]:
inputs = efficientnet.input
x = efficientnet.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

In [38]:
model=tf.keras.Model(inputs=inputs,outputs=outputs)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 rescaling_2 (Rescaling)        (None, None, None,   0           ['input_2[0][0]']                
                                3)                                                                
                                                                                                  
 normalization_1 (Normalization  (None, None, None,   7          ['rescaling_2[0][0]']            
 )                              3)                                                            

In [39]:
hist = model.fit(x_train, y_train, epochs=5, verbose=1)

Epoch 1/5
1/1 [==============================] - 53s 53s/step - loss: 1.1736 - accuracy: 0.2667
Epoch 2/5
1/1 [==============================] - 0s 190ms/step - loss: 0.1490 - accuracy: 1.0000
Epoch 3/5
1/1 [==============================] - 0s 155ms/step - loss: 0.0321 - accuracy: 1.0000
Epoch 4/5
1/1 [==============================] - 0s 152ms/step - loss: 0.0103 - accuracy: 1.0000
Epoch 5/5
1/1 [==============================] - 0s 152ms/step - loss: 0.0059 - accuracy: 1.0000


In [80]:
pred=model.evaluate(x_test,y_test)

1/1 [==============================] - 5s 5s/step - loss: 1.1099 - accuracy: 0.3333
